In [183]:
# %pip install numpy
# %pip install pandas
# %pip install torch        

In [184]:
#installing the required libraries
# %pip install transformers
# %pip install torchtext
# %pip install sentencepiece
# %pip install datasets
# %pip install torchmetrics
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# #huggiface download
# %pip install huggingface_hub


In [185]:
# %pip install llama-cpp-python

In [186]:
from huggingface_hub import hf_hub_download

In [187]:
from llama_cpp import Llama

In [188]:
#download the model llama 2 13b chat gguf
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is in gguf format

In [189]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#print toal path
print(model_path)

/home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


In [190]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )
     

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rop

In [191]:

prompt= f'''Twelve + 3 = ?'''

prompt_template=f'''SYSTEM: You are a math assistant.I will ask you some addition questions. Please answer in word format. Numbers can be in digit or word format. For example, if I ask you 'three + 2 = ?', you should answer 'answer=five'. Do not include any other information in your answer.

USER: {prompt}
        
ASSISTANT:'''
        

In [192]:
response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)


llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =       2.12 ms /     5 runs   (    0.42 ms per token,  2354.05 tokens per second)
llama_print_timings: prompt eval time =    3624.20 ms /    74 tokens (   48.98 ms per token,    20.42 tokens per second)
llama_print_timings:        eval time =    2710.36 ms /     4 runs   (  677.59 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    6349.75 ms /    78 tokens


In [193]:
print(response["choices"][0]["text"])

 Answer=fifteen


In [194]:
import json

In [195]:
#load file in data folder int_addition.json
with open('numeration_int_addn.json') as f:
    data = json.load(f)

In [196]:
#data as pandas dataframe
import pandas as pd

gf = pd.DataFrame(data)
#last 5 rows
gf.tail()

,both positive,word neg digit pos,both negative
dw,"{'d1': [[0, 'zero', 'zero', 0], [0, 'one', 'on...","{'d1': [[0, 'minus one', 'minus one', -1], [0,...","{'d1': [[-1, 'minus one', 'minus two', -2], [-..."
ww,"{'d1': [['zero', 'zero', 'zero', 0], ['zero', ...","{'d1': [['zero', 'minus one', 'minus one', -1]...","{'d1': [['minus one', 'minus one', 'minus two'..."


In [197]:
def extract_data(column_name,data_type):
    df=pd.DataFrame(gf[column_name][data_type]['d1'],columns=['a','b','sum_word','sum_digit'])
    for i in range(2,8):
        df=pd.concat([df,pd.DataFrame(gf[column_name][data_type][f'd{i}'],columns=['a','b','sum_word','sum_digit'])],ignore_index=False)

    return df

df=extract_data('word neg digit pos','dw')

In [198]:
print(df.head())

             a           b     sum_word  sum_digit
0            0   minus one    minus one         -1
1            0   minus two    minus two         -2
2  minus three           0  minus three         -3
3   minus four           0   minus four         -4
4            0  minus five   minus five         -5


In [199]:
print(len(df))

990


In [200]:
import time

#run for list_size_3_pos first column of datafram df

for i in range(0, len(df)):
    
    if i%100==0:
        print(f"Completed {i} rows")
        
    try:
        #prompt is f"Find the minimum number in the list [2, 0, 4, 5]"
        prompt= f"{df.iloc[i, 0]} + {df.iloc[i, 1]} = ?"


        prompt_template=f'''SYSTEM: You are a math assistant.I will ask you some addition questions. Please answer in word format. Numbers can be in digit or word format. For example, if I ask you 'three + 2 = ?', you should answer 'answer=five'. Do not include any other information in your answer.
        
        USER: {prompt}
                
        ASSISTANT:'''
        
        response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=False)
        
        with open('numertion_int_addn_wndp_dw_ow.txt', 'a') as f:
            answer=response["choices"][0]["text"]
            f.write(f"{df.iloc[i, 0]} + {df.iloc[i, 1]} = {answer} \n")
            
            
    except:
        #write in min_output.txt'
        with open('numertion_int_addn_wndp_dw_ow.txt', 'a') as f:
            f.write(f"{df.iloc[i, 0]} + {df.iloc[i, 1]} = Error \n")
            
        time.sleep(2)

Completed 0 rows


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =       1.70 ms /     4 runs   (    0.42 ms per token,  2355.71 tokens per second)
llama_print_timings: prompt eval time =    2386.82 ms /    48 tokens (   49.73 ms per token,    20.11 tokens per second)
llama_print_timings:        eval time =    2024.00 ms /     3 runs   (  674.67 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    4422.62 ms /    51 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =       1.65 ms /     4 runs   (    0.41 ms per token,  2425.71 tokens per second)
llama_print_timings: prompt eval time =     659.75 ms /    12 tokens (   54.98 ms per token,    18.19 tokens per second)
llama_print_timings:        eval time =    2009.88 ms /     3 runs   (  669.96 ms per token,     1.49 tokens per second)
llama_print_timings:       to

Completed 100 rows



llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =       3.56 ms /     8 runs   (    0.45 ms per token,  2244.04 tokens per second)
llama_print_timings: prompt eval time =    1057.06 ms /    20 tokens (   52.85 ms per token,    18.92 tokens per second)
llama_print_timings:        eval time =    4707.72 ms /     7 runs   (  672.53 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =    5787.43 ms /    27 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =       2.16 ms /     5 runs   (    0.43 ms per token,  2311.60 tokens per second)
llama_print_timings: prompt eval time =    1092.51 ms /    20 tokens (   54.63 ms per token,    18.31 tokens per second)
llama_print_timings:        eval time =    2698.01 ms /     4 runs   (  674.50 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    3804.20 ms /    24 

Completed 200 rows


...


llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =       8.81 ms /    20 runs   (    0.44 ms per token,  2269.89 tokens per second)
llama_print_timings: prompt eval time =    1714.54 ms /    34 tokens (   50.43 ms per token,    19.83 tokens per second)
llama_print_timings:        eval time =   12827.38 ms /    19 runs   (  675.13 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   14603.47 ms /    53 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =      13.70 ms /    30 runs   (    0.46 ms per token,  2190.26 tokens per second)
llama_print_timings: prompt eval time =    1756.57 ms /    35 tokens (   50.19 ms per token,    19.93 tokens per second)
llama_print_timings:        eval time =   19472.41 ms /    29 runs   (  671.46 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   21327.35 ms /    64 

Completed 800 rows



llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =      19.23 ms /    47 runs   (    0.41 ms per token,  2444.61 tokens per second)
llama_print_timings: prompt eval time =    1789.68 ms /    34 tokens (   52.64 ms per token,    19.00 tokens per second)
llama_print_timings:        eval time =   30749.09 ms /    46 runs   (  668.46 ms per token,     1.50 tokens per second)
llama_print_timings:       total time =   32684.83 ms /    80 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =      11.30 ms /    26 runs   (    0.43 ms per token,  2301.50 tokens per second)
llama_print_timings: prompt eval time =    1548.09 ms /    31 tokens (   49.94 ms per token,    20.02 tokens per second)
llama_print_timings:        eval time =   16841.82 ms /    25 runs   (  673.67 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   18470.40 ms /    56 

Completed 900 rows



llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =      10.67 ms /    24 runs   (    0.44 ms per token,  2249.51 tokens per second)
llama_print_timings: prompt eval time =    1958.42 ms /    39 tokens (   50.22 ms per token,    19.91 tokens per second)
llama_print_timings:        eval time =   15423.72 ms /    23 runs   (  670.60 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   17458.58 ms /    62 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3624.45 ms
llama_print_timings:      sample time =      23.87 ms /    57 runs   (    0.42 ms per token,  2388.43 tokens per second)
llama_print_timings: prompt eval time =    1957.32 ms /    40 tokens (   48.93 ms per token,    20.44 tokens per second)
llama_print_timings:        eval time =   37550.65 ms /    56 runs   (  670.55 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   39687.32 ms /    96 